In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
# If you run this notebook on colab, please change the notebook path blow to your own drive address.
noteboook_path = '/content/drive/MyDrive/'
os.chdir(noteboook_path) 

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue May 24 16:48:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


## Import packages

In [5]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.0 MB/s 
     |████████████████████████████████| 84 kB 4.3 MB/s 
     |████████████████████████████████| 596 kB 76.6 MB/s 
     |████████████████████████████████| 6.6 MB 56.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
import torch
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import copy
import torch.optim as optim
import random
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup, AutoModelForSequenceClassification


## Fetch top k sentences form main text

In [6]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 3.1 MB/s 
     |████████████████████████████████| 1.2 MB 27.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=36212c870554f2ac23458e59a80349213252ff9589e9b7b549c92e722f34e6bc
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


In [ ]:
import nltk
import heapq
from sentence_transformers import SentenceTransformer, util
import pandas as pd

In [7]:
df_train = pd.read_csv('Data/clean_train.tsv', sep='\t')
df_val =  pd.read_csv('Data/clean_dev.tsv', sep='\t')
df_test =  pd.read_csv('Data/clean_test.tsv', sep='\t')

In [ ]:
nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
df_train['sentences'] = df_train.apply(lambda x: tokenizer.tokenize(x['main_text']), axis=1)
df_val['sentences'] = df_val.apply(lambda x: tokenizer.tokenize(x['main_text']), axis=1)
df_test['sentences'] = df_test.apply(lambda x: tokenizer.tokenize(x['main_text']), axis=1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def get_top_k_sentences_text(claim, main, k):
    claim_representation = model.encode(claim, convert_to_tensor=True)
    main_representation = model.encode(main, convert_to_tensor=True)
    similarity = util.pytorch_cos_sim(claim_representation, main_representation).squeeze(0).tolist()
    topk = heapq.nlargest(5, range(len(similarity)), similarity.__getitem__)
    output = ''
    for i in topk:
      output += main[i] + " "
    return  output

In [ ]:
model = SentenceTransformer('bert-base-nli-mean-tokens')
df_train['topk_text'] = df_train.apply(lambda x: get_top_k_sentences_text(x['claim'], x['sentences'], 5), axis=1)
df_val['topk_text'] = df_val.apply(lambda x: get_top_k_sentences_text(x['claim'], x['sentences'], 5), axis=1)
df_test['topk_text'] = df_test.apply(lambda x: get_top_k_sentences_text(x['claim'], x['sentences'], 5), axis=1)

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
df_train.to_csv('Data/clean_train_topk.tsv', sep='\t', index=False)
df_val.to_csv('Data/clean_dev_topk.tsv', sep='\t', index=False)
df_test.to_csv('Data/clean_test_topk.tsv', sep='\t', index=False)

## Import data

In [7]:
df_train = pd.read_csv('Data/clean_train_topk.tsv', sep='\t')
df_val =  pd.read_csv('Data/clean_dev_topk.tsv', sep='\t')
df_test =  pd.read_csv('Data/clean_test_topk.tsv', sep='\t')

In [8]:
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(9804, 5)
(1214, 5)
(1233, 5)


In [9]:
df_train.head()

,label,claim,main_text,sentences,topk_text
0,0,"""The money the Clinton Foundation took from fr...","""Hillary Clinton is in the political crosshair...","['""Hillary Clinton is in the political crossha...",A clause in the Constitution does prohibit U.S...
1,1,Annual Mammograms May Have More False-Positives,While the financial costs of screening mammogr...,['While the financial costs of screening mammo...,Part of the continuing controversy over screen...
2,1,SBRT Offers Prostate Cancer Patients High Canc...,The news release quotes lead researcher Robert...,['The news release quotes lead researcher Robe...,Some of the men in the study had a low-risk fo...
3,2,"Study: Vaccine for Breast, Ovarian Cancer Has ...","The story does discuss costs, but the framing ...","['The story does discuss costs, but the framin...","It says, that the vaccine is “for breast and o..."
4,2,Some appendicitis cases may not require ’emerg...,"""Although the story didn’t cite the cost of ap...","['""Although the story didn’t cite the cost of ...",There were no claims made about the novelty of...


In [10]:
possible_labels = df_train.label.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{0: 0, 1: 1, 2: 2, 3: 3}

In [11]:
df_train['label'] = df_train.label.replace(label_dict)
df_val['label'] = df_val.label.replace(label_dict)
df_test['label'] = df_test.label.replace(label_dict)

## Define BERT Model

In [12]:
class CustomDataset(Dataset):

    def __init__(self, data, maxlen, with_labels=True, bert_model='albert-base-v2'):

        self.data = data  # pandas dataframe
        #Initialize the tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(bert_model)  

        self.maxlen = maxlen
        self.with_labels = with_labels 

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):

        # Selecting sentence1 and sentence2 at the specified index in the data frame
        sent1 = str(self.data.loc[index, 'claim'])
        # sent2 = str(self.data.loc[index, 'main_text'])
        sent2 = str(self.data.loc[index, 'topk_text'])

        # Tokenize the pair of sentences to get token ids, attention masks and token type ids
        encoded_pair = self.tokenizer(sent1, sent2, 
                                      padding='max_length',  # Pad to max_length
                                      truncation=True,  # Truncate to max_length
                                      max_length=self.maxlen,  
                                      return_tensors='pt')  # Return torch.Tensor objects
        
        token_ids = encoded_pair['input_ids'].squeeze(0)  # tensor of token ids
        attn_masks = encoded_pair['attention_mask'].squeeze(0)  # binary tensor with "0" for padded values and "1" for the other values
        token_type_ids = encoded_pair['token_type_ids'].squeeze(0)  # binary tensor with "0" for the 1st sentence tokens & "1" for the 2nd sentence tokens

        if self.with_labels:  # True if the dataset has labels
            label = self.data.loc[index, 'label']
            return token_ids, attn_masks, token_type_ids, label  
        else:
            return token_ids, attn_masks, token_type_ids

In [21]:
class SentencePairClassifier(nn.Module):

    def __init__(self, bert_model='bert-base-uncased', freeze_bert=False, embedding_size=768, dropout_rate=0):
        super(SentencePairClassifier, self).__init__()
        self.bert_model = bert_model
        if bert_model == 'microsoft/deberta-v3-base':
            self.bert_layer = AutoModelForSequenceClassification.from_pretrained(bert_model, num_labels=4, problem_type="multi_label_classification")

        #  Instantiating BERT-based model object
        else:
            self.bert_layer = AutoModel.from_pretrained(bert_model)

            # Freeze bert layers and only train the classification layer weights
            if freeze_bert:
                for p in self.bert_layer.parameters():
                    p.requires_grad = False

            # Classification layer
            self.cls_layer = nn.Linear(embedding_size, len(label_dict))

            self.dropout = nn.Dropout(p=dropout_rate)

    def forward(self, input_ids, attn_masks, token_type_ids):
        '''
        Inputs:
            -input_ids : Tensor  containing token ids
            -attn_masks : Tensor containing attention masks to be used to focus on non-padded values
            -token_type_ids : Tensor containing token type ids to be used to identify sentence1 and sentence2
        '''

        
        if self.bert_model == 'microsoft/deberta-v3-base':
            loss = self.bert_layer(input_ids, attn_masks, token_type_ids).logits
            return loss
            
        else:
            # Feeding the inputs to the BERT-based model to obtain contextualized representations
            pooler_output = self.bert_layer(input_ids, attn_masks, token_type_ids)['pooler_output']
            # Feeding to the classifier layer the last layer hidden-state of the [CLS] token further processed by a
            # Linear Layer and a softmax activation. 

            logits = self.cls_layer(pooler_output)
        return logits

In [22]:
def set_seed(seed):
    """ Set all seeds to make results reproducible """
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    

def evaluate_loss(net, device, criterion, dataloader):
    net.eval()

    mean_loss = 0
    count = 0

    with torch.no_grad():
        for it, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm(dataloader)):
            seq, attn_masks, token_type_ids, labels = \
                seq.to(device), attn_masks.to(device), token_type_ids.to(device), labels.to(device)
            logits = net(seq, attn_masks, token_type_ids)
            mean_loss += criterion(logits, labels).item()
            count += 1

    return mean_loss / count

In [23]:
print("Creation of the models' folder...")
!mkdir models

Creation of the models' folder...
mkdir: cannot create directory ‘models’: File exists


In [25]:
def train_bert(net, criterion, opti, lr, train_loader, val_loader, epochs):

    best_loss = np.Inf
    best_ep = 1
    nb_iterations = len(train_loader)
    print_every = nb_iterations // 5  # print the training loss 5 times per epoch


    for ep in range(epochs):

        net.train()
        for it, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm(train_loader)):

            # Converting to cuda tensors
            seq, attn_masks, token_type_ids, labels = \
                seq.to(device), attn_masks.to(device), token_type_ids.to(device), labels.to(device)
    
            logits = net(seq, attn_masks, token_type_ids)
            loss = criterion(logits, labels)
            opti.zero_grad()
            loss.backward()
            opti.step()




        val_loss = evaluate_loss(net, device, criterion, val_loader)  # Compute validation loss
        print()
        print("Epoch {} complete! Validation Loss : {}".format(ep+1, val_loss))

        if val_loss < best_loss:
            print("Best validation loss improved from {} to {}".format(best_loss, val_loss))
            print()
            net_copy = copy.deepcopy(net)  # save a copy of the model
            best_loss = val_loss
            best_ep = ep + 1

    # Saving the model
    path_to_model='models/{}_lr_{}_val_loss_{}_ep_{}.pt'.format(bert_model.replace('/', '-'), lr, round(best_loss, 5), best_ep)
    torch.save(net_copy.state_dict(), path_to_model)
    print("The model has been saved in {}".format(path_to_model))

    del loss
    torch.cuda.empty_cache()

## Focal Loss

In [22]:
##Define focal loss function
from torch import nn
import torch
from torch.nn import functional as F
class focal_loss(nn.Module):    
    def __init__(self, alpha=0.25, gamma=2, num_classes = 4, size_average=True):
        

        super(focal_loss,self).__init__()
        self.size_average = size_average
        if isinstance(alpha,list):
            assert len(alpha)==num_classes   
            print("Focal_loss alpha = {},".format(alpha))
            self.alpha = torch.Tensor(alpha)
        else:
            assert alpha<1   
            print(" --- Focal_loss alpha = {} ".format(alpha))
            self.alpha = torch.zeros(num_classes)
            self.alpha[0] += alpha
            self.alpha[1:] += (1-alpha) 
        self.gamma = gamma

    def forward(self, preds, labels):
  
        # assert preds.dim()==2 and labels.dim()==1        
        preds = preds.view(-1,preds.size(-1))        
        self.alpha = self.alpha.to(preds.device)        
        preds_softmax = F.softmax(preds, dim=1)        
        preds_logsoft = torch.log(preds_softmax)
        preds_softmax = preds_softmax.gather(1,labels.view(-1,1))   # nll_loss ( crossempty = log_softmax + nll )        
        preds_logsoft = preds_logsoft.gather(1,labels.view(-1,1))        
        self.alpha = self.alpha.gather(0,labels.view(-1))        
        loss = -torch.mul(torch.pow((1-preds_softmax), self.gamma), preds_logsoft)  # torch.pow((1-preds_softmax), self.gamma) 
        loss = torch.mul(self.alpha, loss.t())        
        if self.size_average:        
            loss = loss.mean()        
        else:            
            loss = loss.sum()        
        return loss

## Train model

In [26]:
# bert_model = 'bert-base-uncased' 
# bert_model = 'allenai/scibert_scivocab_uncased'
# bert_model =  "albert-base-v2"
# bert_model = "google/bigbird-roberta-base"
bert_model = 'microsoft/deberta-v3-base'


freeze_bert = False  # if True, freeze the encoder weights and only update the classification layer weights
maxlen = 512  # maximum length of the tokenized input sentence pair : if greater than "maxlen", the input is truncated and else if smaller, the input is padded
bs = 8  # batch size
lr = 1e-6  # learning rate
epochs = 4  # number of training epochs
embedding_size = 768 # Embedding size for encoder output

In [27]:
# If you encounter a CUDA out of memory error: 
# - uncomment the kill command, run the "kill" command (and comment it)
# - reduce the batch size
# - then run all cells from the begining 

# If you get an ugly print of tqdm (all iterations are showed), follow the above first and last steps

# printm()
# !kill -9 -1

In [28]:
#  Set all seeds to make reproducible results
set_seed(24)
# Creating instances of training and validation set
print("Reading training data...")
train_set = CustomDataset(df_train, maxlen, bert_model)
print("Reading validation data...")
val_set = CustomDataset(df_val, maxlen, bert_model)
# Creating instances of training and validation dataloaders
train_loader = DataLoader(train_set, batch_size=bs, num_workers=4)
val_loader = DataLoader(val_set, batch_size=bs, num_workers=4)


device = torch.device("cuda:0")

net = SentencePairClassifier(bert_model, freeze_bert=freeze_bert, embedding_size=embedding_size)

if torch.cuda.device_count() > 1:  # if multiple GPUs
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    net = nn.DataParallel(net)

net.to(device)

#cross entropy
criterion = nn.CrossEntropyLoss()

# focal_loss
# cat = df_train.groupby('label').size()
# weights = list(1/(cat/np.median(cat)))
# criterion = focal_loss(weights)

opti = AdamW(net.parameters(), lr=lr, weight_decay=1e-2)

train_bert(net, criterion, opti, lr, train_loader, val_loader, epochs)

Reading training data...
Reading validation data...


Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifier.weight', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a


Epoch 1 complete! Validation Loss : 1.0677518687750165
Best validation loss improved from inf to 1.0677518687750165



100%|██████████| 152/152 [00:35<00:00,  4.24it/s]



Epoch 2 complete! Validation Loss : 1.0118402113255702
Best validation loss improved from 1.0677518687750165 to 1.0118402113255702



100%|██████████| 152/152 [00:36<00:00,  4.22it/s]



Epoch 3 complete! Validation Loss : 0.9669990118004774
Best validation loss improved from 1.0118402113255702 to 0.9669990118004774



100%|██████████| 152/152 [00:35<00:00,  4.25it/s]



Epoch 4 complete! Validation Loss : 0.9447986299270078
Best validation loss improved from 0.9669990118004774 to 0.9447986299270078

The model has been saved in models/microsoft-deberta-v3-base_lr_1e-06_val_loss_0.9448_ep_4.pt


##Get Prediction

In [29]:
print("Creation of the results' folder...")
!mkdir results

Creation of the results' folder...
mkdir: cannot create directory ‘results’: File exists


In [30]:
def get_probs_from_logits(logits):
    """
    Converts a tensor of logits into an array of probabilities by applying the sigmoid function
    """
    probs = torch.softmax(logits, dim=1)
    return probs.detach().cpu().numpy()

def test_prediction(net, device, dataloader, with_labels=True, result_file="results/output.txt"):
    """
    Predict the probabilities on a dataset with or without labels and print the result in a file
    """
    net.eval()
    w = open(result_file, 'w')
    probs_all = []

    with torch.no_grad():
        if with_labels:
            for seq, attn_masks, token_type_ids, _ in tqdm(dataloader):
                seq, attn_masks, token_type_ids = seq.to(device), attn_masks.to(device), token_type_ids.to(device)
                logits = net(seq, attn_masks, token_type_ids)
                probs = get_probs_from_logits(logits)
                probs_all += probs.tolist()
        else:
            for seq, attn_masks, token_type_ids in tqdm(dataloader):
                seq, attn_masks, token_type_ids = seq.to(device), attn_masks.to(device), token_type_ids.to(device)
                logits = net(seq, attn_masks, token_type_ids)
                probs = get_probs_from_logits(logits)
                probs_all += probs.tolist()

    w.writelines(str(np.argmax(prob)) + '\n' for prob in probs_all)
    w.close()

In [33]:
# bert_model = 'bert-base-uncased'
# path_to_model = 'models/bert-base-uncased_lr_1e-06_val_loss_0.97006_ep_4.pt' #bert-ce-loss-all-sentences
# path_to_model = 'models/bert-base-uncased_lr_1e-06_val_loss_0.95399_ep_3.pt' #bert-ce-loss
# path_to_model = 'models/bert-base-uncased_lr_1e-06_val_loss_0.34906_ep_4.pt' #bert-focal-loss
# bert_model = 'allenai/scibert_scivocab_uncased'
# path_to_model = 'models/allenai-scibert_scivocab_uncased_lr_1e-06_val_loss_0.81592_ep_4.pt' #scibert-ce-loss
# bert_model =  "albert-base-v2"
# path_to_model = 'models/albert-base-v2_lr_1e-06_val_loss_0.71444_ep_3.pt' #albert-ce-loss
# bert_model = "google/bigbird-roberta-base"
# path_to_model = 'models/google-bigbird-roberta-base_lr_1e-06_val_loss_0.91867_ep_4.pt'
bert_model = 'microsoft/deberta-v3-base'
path_to_model = 'models/microsoft-deberta-v3-base_lr_1e-06_val_loss_0.9448_ep_4.pt'

In [34]:
path_to_output_file = 'results/output_{}.txt'.format(path_to_model.replace('/','-'))
device = torch.device("cuda:0")
print("Reading test data...")
test_set = CustomDataset(df_test, maxlen, bert_model)
test_loader = DataLoader(test_set, batch_size=bs, num_workers=4)

model = SentencePairClassifier(bert_model)
if torch.cuda.device_count() > 1:  # if multiple GPUs
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)

print()
print("Loading the weights of the model...")
model.load_state_dict(torch.load(path_to_model))
model.to(device)

print("Predicting on test data...")
test_prediction(net=model, device=device, dataloader=test_loader, with_labels=True, result_file=path_to_output_file)
print()
print("Predictions are available in : {}".format(path_to_output_file))

Reading test data...


Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifier.weight', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a


Loading the weights of the model...
Predicting on test data...


100%|██████████| 155/155 [00:36<00:00,  4.24it/s]


Predictions are available in : results/output_models-microsoft-deberta-v3-base_lr_1e-06_val_loss_0.9448_ep_4.pt.txt


## Evaluate

###BERT

#### With all sentences

In [35]:
path_to_model = 'models/bert-base-uncased_lr_1e-06_val_loss_0.97006_ep_4.pt' #bert-ce-loss-all-sentences
path_to_output_file = 'results/output_{}.txt'.format(path_to_model.replace('/', '-'))  
labels_test = df_test['label']  # true labels
preds_test = pd.read_csv(path_to_output_file, header=None)[0]  
print(classification_report(labels_test, preds_test))

              precision    recall  f1-score   support

           0       0.50      0.33      0.40       388
           1       0.00      0.00      0.00       201
           2       0.55      0.90      0.69       599
           3       0.00      0.00      0.00        45

    accuracy                           0.54      1233
   macro avg       0.26      0.31      0.27      1233
weighted avg       0.43      0.54      0.46      1233



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


####With topk-sentences and cross-entropy loss

In [36]:
path_to_model = 'models/bert-base-uncased_lr_1e-06_val_loss_0.95399_ep_3.pt' #bert-ce-loss
path_to_output_file = 'results/output_{}.txt'.format(path_to_model.replace('/', '-'))  
labels_test = df_test['label']  # true labels
preds_test = pd.read_csv(path_to_output_file, header=None)[0]  
print(classification_report(labels_test, preds_test))

              precision    recall  f1-score   support

           0       0.51      0.54      0.53       388
           1       0.00      0.00      0.00       201
           2       0.62      0.85      0.72       599
           3       0.00      0.00      0.00        45

    accuracy                           0.58      1233
   macro avg       0.28      0.35      0.31      1233
weighted avg       0.46      0.58      0.51      1233



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


###With topk-sentences and focal loss

In [37]:
path_to_model = 'models/bert-base-uncased_lr_1e-06_val_loss_0.34906_ep_4.pt' #bert-focal-loss
path_to_output_file = 'results/output_{}.txt'.format(path_to_model.replace('/', '-'))  
labels_test = df_test['label']  # true labels
preds_test = pd.read_csv(path_to_output_file, header=None)[0]  
print(classification_report(labels_test, preds_test))

              precision    recall  f1-score   support

           0       0.51      0.52      0.52       388
           1       0.00      0.00      0.00       201
           2       0.61      0.85      0.71       599
           3       0.00      0.00      0.00        45

    accuracy                           0.58      1233
   macro avg       0.28      0.34      0.31      1233
weighted avg       0.46      0.58      0.51      1233



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


###SciBERT--Pretrained on specific domain

In [38]:
path_to_model = 'models/allenai-scibert_scivocab_uncased_lr_1e-06_val_loss_0.81592_ep_4.pt' #scibert-ce-loss
path_to_output_file = 'results/output_{}.txt'.format(path_to_model.replace('/', '-'))  
labels_test = df_test['label']  # true labels
preds_test = pd.read_csv(path_to_output_file, header=None)[0]  
print(classification_report(labels_test, preds_test))

              precision    recall  f1-score   support

           0       0.49      0.81      0.61       388
           1       0.00      0.00      0.00       201
           2       0.76      0.75      0.76       599
           3       0.00      0.00      0.00        45

    accuracy                           0.62      1233
   macro avg       0.31      0.39      0.34      1233
weighted avg       0.52      0.62      0.56      1233



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


###ALBERT

In [39]:
path_to_model = 'models/albert-base-v2_lr_1e-06_val_loss_0.71444_ep_3.pt' #albert-ce-loss
path_to_output_file = 'results/output_{}.txt'.format(path_to_model.replace('/', '-'))  
labels_test = df_test['label']  # true labels
preds_test = pd.read_csv(path_to_output_file, header=None)[0]  
print(classification_report(labels_test, preds_test))

              precision    recall  f1-score   support

           0       0.51      0.86      0.64       388
           1       0.57      0.06      0.11       201
           2       0.82      0.76      0.79       599
           3       0.00      0.00      0.00        45

    accuracy                           0.65      1233
   macro avg       0.47      0.42      0.38      1233
weighted avg       0.65      0.65      0.60      1233



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


###BigBird

In [40]:
path_to_model = 'models/google-bigbird-roberta-base_lr_1e-06_val_loss_0.91867_ep_4.pt'
path_to_output_file = 'results/output_{}.txt'.format(path_to_model.replace('/', '-'))  
labels_test = df_test['label']  # true labels
preds_test = pd.read_csv(path_to_output_file, header=None)[0]  
print(classification_report(labels_test, preds_test))

              precision    recall  f1-score   support

           0       0.46      0.70      0.55       388
           1       1.00      0.00      0.01       201
           2       0.69      0.74      0.71       599
           3       0.00      0.00      0.00        45

    accuracy                           0.58      1233
   macro avg       0.54      0.36      0.32      1233
weighted avg       0.64      0.58      0.52      1233



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


###DeBERTaV3

In [42]:
path_to_model = 'models/microsoft-deberta-v3-base_lr_1e-06_val_loss_0.9448_ep_4.pt'
path_to_output_file = 'results/output_{}.txt'.format(path_to_model.replace('/', '-'))  
labels_test = df_test['label']  # true labels
preds_test = pd.read_csv(path_to_output_file, header=None)[0]  
print(classification_report(labels_test, preds_test))

              precision    recall  f1-score   support

           0       0.47      0.55      0.51       388
           1       0.00      0.00      0.00       201
           2       0.63      0.82      0.71       599
           3       0.00      0.00      0.00        45

    accuracy                           0.57      1233
   macro avg       0.28      0.34      0.30      1233
weighted avg       0.45      0.57      0.51      1233



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
